In [ ]:
#!tar chvfz notebook.tar.gz *

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Remove chain setting warning
import numpy as np
import missingno as msno
import plotly as py
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
from optuna.integration import LightGBMPruningCallback
from optuna.samplers import TPESampler
from geopy.geocoders import Nominatim
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
import xgboost as xgb
import lightgbm as lgbm # Ok, Billy you need to deal with this somehow ... I guess getting access to a GPU machine will work
import time

r_s = 0

## Load in data

In [ ]:
# Load in housing data

house_data = pd.read_csv("housing_in_london/housing_in_london_yearly_variables.csv") #billy you need to cut sub 2001 data?
house_data.rename(columns={"area": "borough_name"}, inplace=True)
house_data.borough_name = house_data.borough_name.str.title()
house_data.borough_name = house_data.borough_name.replace(['Of','And'],['of','and'], regex=True)
house_data.dtypes # Lets check datatypes. FOr some reason mean salary and recylcing percentages are objects not floats. Lets change that!
house_data.mean_salary = house_data.mean_salary.apply(pd.to_numeric, errors='coerce') #Drop strings and make float
house_data.recycling_pct = house_data.recycling_pct.apply(pd.to_numeric, errors='coerce') #Drop strings and make float


house_data.borough_name.unique() #Some of these aren't London boroughs

remove_boroughs = ['North East',
       'North West', 'Yorkshire and The Humber', 'East Midlands',
       'West Midlands', 'East', 'South East', 'South West',
       'Inner London', 'Outer London', 'United Kingdom', 'Great Britain',
       'England and Wales', 'Northern Ireland', 'Scotland', 'Wales', 'England', 'London']

house_data = house_data[~house_data["borough_name"].isin(remove_boroughs)]

# Change dates to just years
house_data['year'] = house_data['date'].str.split('-').str[0]
house_data.drop(columns = ["date", "borough_flag"], inplace = True)


msno.matrix(house_data)



In [ ]:
# Let's do visualisation for the two datasets seperately

In [ ]:

# FInally, we need to add an "election" column

def add_election(row):
    if int(row.year) < 2006:
        election = "2005"
        return election
    elif int(row.year) < 2011:
        election = "2010"
        return election
    elif int(row.year) < 2016:
        election = "2015"
        return election
    elif int(row.year) < 2018:
        election = "2017"
        return election
    else:
        election = "2019"
        return election
        
house_data["election"] = house_data.apply(lambda row: add_election(row), axis=1)


msno.matrix(house_data)
#Done

In [ ]:

# Let's first drop the entire "life_satisfaction" column here as it is missing lots of values. We can then remove individual rows that contain a "NaN.
house_data.drop(columns = "life_satisfaction", inplace = True)

house_data.hist(figsize=(15,15))

#All the features with missing values skew, so let's try replacing all with median
house_data.fillna(house_data.median(),inplace=True)
plt.show()

#house_data = house_data.fillna(house_data.groupby('borough_name').transform('mean'))

msno.matrix(house_data)

Load in election data

In [ ]:
# Load in election data

election_data = pd.read_csv("1918-2019election_results.csv",encoding = "ISO-8859-1") #Needs encoding specified or wont read file
election_data.rename(columns={'lib_votes ':'lib_votes'}, inplace=True)
election_data = election_data[election_data["election"]>"2001"] #this now matches other data
election_data = election_data[election_data["country/region"]=='London'] #Only London
election_data = election_data.astype({'lib_votes': float, 'lab_votes': float})
election_data = election_data[["constituency_id", "constituency_name", "con_votes", "lib_votes" ,"lab_votes", "election"]] # We want the share (percentage of votes rather than raw number)
election_data.constituency_name = election_data.constituency_name.replace(['&','And'],'and', regex=True)
election_data.constituency_name = election_data.constituency_name.replace('Of','of', regex=True)
#Load in constituency list for conversion

constituencies = pd.read_excel("PCON_DEC_2021_UK_NC.xlsx", sheet_name="PCON_DEC_2021_UK_NC", names=["constituency_code", "constituency_name"])
constituencies.constituency_name = constituencies.constituency_name.replace(',','', regex=True)
dict_name_code = dict(zip(constituencies.constituency_name,constituencies.constituency_code))

# Replace old constituency with closest modern analogue

dict_replace_old_constituencies = {'Brent East':'Chelsea and Fulham', #Billy this line is a fudge, but come back to it and fix it later
 'Brent South':'Brent Central',
 'Dagenham':'Dagenham and Rainham',
 "Ealing Acton and Shepherd'S Bush":'Ealing Central and Acton',
 'Hammersmith and Fulham':'Hammersmith',
 'Hampstead and Highgate':'Hampstead and Kilburn',
 'Hornchurch':'Hornchurch and Upminster',
 'Kensington and Chelsea':'Kensington',
 'Lewisham West':'Lewisham West and Penge',
 'North Southwark and Bermondsey':'Bermondsey and Old Southwark',
 'Poplar and Canning Town':'Poplar and Limehouse',
 "Regent'S Park and Kensington North":'Westminster North',
 'Ruislip Northwood':'Ruislip Northwood and Pinner',
 'Upminster':'Hornchurch and Upminster',
 'Uxbridge':'Uxbridge and South Ruislip'}

election_data.replace({"constituency_name" : dict_replace_old_constituencies},inplace=True)

# Remap to account for different or out-of-date codes

election_data.constituency_id = election_data.constituency_name.map(dict_name_code)

msno.matrix(election_data)

In [ ]:
election_data.hist(figsize=(15,15))
plt.show()

In [ ]:
# lib_votes distribution also skews, so use median to replace

election_data.fillna(election_data.median(),inplace=True)

In [ ]:


#Determine winner by year

def determine_winner(row):
    vote_options = [row.con_votes, row.lib_votes, row.lab_votes]
    win = np.argmax(vote_options)
    if win == 0:
        winner = "con"
    elif win == 1:
        winner = "lib"
    elif win == 2:
        winner = "lab"
    return winner
        
election_data["winner"] = election_data.apply(lambda row: determine_winner(row), axis=1)



In [ ]:
constituencies_boroughs = pd.read_csv("constituencies_boroughs.csv")
dict_constituencies_boroughs = dict(zip(constituencies_boroughs.Constituency,constituencies_boroughs.Borough))

election_data["borough_name"]=election_data["constituency_name"].map(dict_constituencies_boroughs)

In [ ]:
combined_data = pd.merge(election_data, house_data, how="inner", on=["borough_name", "election"])
msno.matrix(combined_data)

## Potential visualisation section

In [ ]:
ax = sns.countplot(combined_data.winner, label="Count",  palette=["#FF0000","#00008B", "#FFFF00"])    
La,Co,Li = combined_data.winner.value_counts()
print('Number of Labour: ',La)
print('Number of Conservative : ',Co)
print('Number of Liberal Democrat : ',Li)

In [ ]:
combined_data.hist(figsize=(15,15))
plt.show()

In [ ]:
combined_data.plot(kind="density", layout=(6,5), 
             subplots=True,sharex=False, sharey=False, figsize=(15,15))
plt.show()

In [ ]:
#We should do some visualisation of "final" prepared dataset for feature selection? or do it earlier? let me think ...

In [ ]:
# You also need to do feature selection, outlier removal etc ...

In [ ]:
data_for_ml = combined_data.drop(columns = ['constituency_name', 'constituency_id', 'con_votes', 'lib_votes', 'lab_votes', 'election',
       'borough_name', 'code', 'year'] ,inplace = False)

In [ ]:
data_for_ml.dtypes
#Split post-2019 and pre-2019 everything pre-2019 election results is train basically
pre = data_for_ml[combined_data.election != "2019"] #No "election" column in ML dataset, but can use information from original combined_dataset
post = data_for_ml[combined_data.election == "2019"]
# These are all numerical float variables
# https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/
# According to this, use ANOVA -> You need to split pre here to do feature selection or youll cross contaminate

In [ ]:
Billy to do:
    Accuracy obtained by predictions (especialy 2019 predicted vs actual) is consistently exactly reproducible. Why? (Have a proper think of what you're actually doing)
    Write some code to auto-select best optuna resuts (normalised) and non normalised, and use accordingly (make sure to normalise the data if necessary!)
    Visualisation/feature selection (even feature creation?) Remember goal is to get cross val accuracy as high as possible (predicted vs actual doesnt need to be as accurate as possible because we want to know how different brexit made things, but the validation results do need to be accurate!)

In [ ]:
X = pre.drop(columns="winner") #wierd indexing problem here. Same as the reset index earlier, not solved though. Annyoing but ok
y = pre.winner

X_pre_feature_select, X_val, y_pre_feature_select, y_val = train_test_split(X, y, stratify=y, random_state=0)


selector = SelectKBest(f_classif, k=4)
select = selector.fit(X_pre_feature_select, y_pre_feature_select)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()
print(scores)

X_indices = np.arange(X_pre_feature_select.shape[-1])
plt.figure(1)
plt.clf()
plt.bar(X_indices - 0.05, scores, width=0.2)
plt.title("Feature univariate score")
plt.xlabel("Feature number")
plt.ylabel(r"Univariate score ($-Log(p_{value})$)")
plt.show()

#REALLY, AREA SIZE IS BIGGEST FACTOR??? MORE THAN SALARY .... HMMM

In [ ]:
selected_features = ['recycling_pct','population_size','area_size','no_of_houses']

## Here is the ML section, you are going to predict the 2019 results and compare them to the real 2019 and see the effect of Brexit

Billy, you need to get this accuracy in the validation section as high as possible (via feature selection, outlier removal etc), because that model will be used to predict 2019. So quality of analysis depnds on this


### ML with normalised data

In [ ]:
X_val = X_val[selected_features]


X_train, X_test, y_train, y_test = train_test_split(X_val, y_val, test_size=0.2, stratify=y_val,random_state=r_s)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#YOU NEED TO NORMALIZE THE DATA HERE BECAUSE OF THE SVM! EVEN THOUGH YOU DONT NEED TO FOR RANDOM FOREST OR ACTUALLY MAYBE SPLIT OPTUNA INTO TREE AND NON-TREE, SO SPLIT BETWEEN DATA THAT NEEDS NORMALIZED AND DOESNT

#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = trial.suggest_categorical("classifier", ["SVC", "KNeighbors"])
    
    # Step 2. Setup values for the hyperparameters:
        
    if classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
        classifier_object = SVC(C=svc_c, gamma="auto")    

    else:
        n_neighbors = trial.suggest_int("n_neighbors", 1, 30)
        weights = trial.suggest_categorical("weights", ['uniform', 'distance'])
        classifier_object = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)

    # Step 3: Scoring method:
    classifier_object.fit(X_train, y_train)
    preds = classifier_object.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

# Step 4: Running it
optuna.logging.set_verbosity(optuna.logging.CRITICAL)
svc_knn_study = optuna.create_study(direction="maximize")
svc_knn_study.optimize(objective, n_trials=1000)
print(svc_knn_study.best_trial)
print(svc_knn_study.best_trial.value)

### No normalisation ML

In [ ]:
# Now for more complex, lightgbm and xgboost

#Start simple, then when you get the hang of it, follow from here

#https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5  (he does CV within the objective call, which is what I wanted to know! So you CAN do it)
#https://towardsdatascience.com/how-to-beat-the-heck-out-of-xgboost-with-lightgbm-comprehensive-tutorial-5eba5219599

#This is the one you ended up basing yourself on: https://practicaldatascience.co.uk/machine-learning/how-to-tune-a-lightgbmclassifier-model-with-optuna

X_val = X_val[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_val, y_val, test_size=0.2, stratify=y_val, random_state=r_s)


def objective(trial):
    """
    Objective function to be minimized.
    """

    classifier_name = trial.suggest_categorical("classifier", ["LightGBM", "RandomForest"])

    if classifier_name == "LightGBM":
    
        param = {
            "objective": "multiclass",
            "metric": "multi_logloss",
            "verbosity": -1, 
            "boosting_type": "gbdt",
            "num_class": 3,
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),}
            
        classifier_object = lgbm.LGBMClassifier(**param)
            
    else:
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        classifier_object = RandomForestClassifier(max_depth=rf_max_depth, n_estimators=rf_n_estimators)
        
        
    classifier_object.fit(X_train, y_train)
    preds = classifier_object.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

optuna.logging.set_verbosity(optuna.logging.CRITICAL)
rf_lgbm_study = optuna.create_study(study_name="rf_lightgbm", direction="maximize")
rf_lgbm_study.optimize(objective, n_trials=1000)
print(rf_lgbm_study.best_trial)
print(rf_lgbm_study.best_trial.value)

## Use best results from validation to predict 2019 results

Best results are obtained using LightGBM, so let's use a LightGBM model. NOT NECESSARILY, WRITE SOME EASY IF CODE TO CHECK THIS

In [ ]:
# pre = data_for_ml[combined_data.election != "2019"] #No "election" column in ML dataset, but can use information from original combined_dataset
# post = data_for_ml[combined_data.election == "2019"]

X_train = pre.drop(columns="winner") #Train on all the pre data (so feature selection data and validation data)
y_train = pre.winner

X_test = post.drop(columns="winner") #Test in all post data

param = rf_lgbm_study.best_params

#classifier_object = lgbm.LGBMClassifier(**param)
classifier_object = RandomForestClassifier(n_estimators= 31, max_depth= 5)


classifier_object.fit(X_train, y_train)
preds = classifier_object.predict(X_test)
#How accurate were my predictions?
accuracy = accuracy_score(y_test, preds)
print(accuracy)

Add predicted results to combined_data frame


In [ ]:
predicted_2019_frame = pd.DataFrame(combined_data[combined_data.election == "2019"])

predicted_2019_frame["winner"] = preds
predicted_2019_frame["election"] = "2019 Predicted"



# Bit of fudging here. Need to rethink classification at some point but for now this makes sense logically. (Only consider actual election year results)

assign_list = []
for group in predicted_2019_frame.groupby('constituency_name'):
    if group[1].winner.iloc[0] != group[1].winner.iloc[1]:
        group[1].winner.iloc[0] = group[1].winner.iloc[1]
    assign_list.append(group[1].winner)

predicted_2019_frame["winner"] = pd.concat(assign_list)
combined_data = pd.concat([combined_data,predicted_2019_frame])

Link the final dataset to the polygon information

In [ ]:
shapefile = gpd.read_file("Westminster_Parliamentary_Constituencies_(Dec_2021)_UK_BUC/Westminster_Parliamentary_Constituencies_(Dec_2021)_UK_BUC.shp")

shapefile.rename(columns={"PCON21CD": "constituency_id"},inplace=True)

print(shapefile)

In [ ]:
geo_combined_data = pd.merge(combined_data, shapefile[["constituency_id", "geometry"]], how="inner", on= "constituency_id")
geo_combined_data.dropna(inplace=True)

In [ ]:
geo_combined_data.drop(columns=["code","borough_flag", "borough_name", "year"])

In [ ]:
geo_combined_data = gpd.GeoDataFrame(geo_combined_data) #Can you get away with only dropping coords here?
geo_combined_data.set_crs("epsg:27700",inplace=True) #This line is importtant, Billy explain what this is and how you obtained it! Basically its encoding infrormation and if you dont include it wont work\n",

geo_combined_data.to_file('LondonFiles.shp', driver='ESRI Shapefile') #not sure we need the driver, check. THIS IS FINAL OUTPUT FOR TABLEAU"